## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import requests
from bs4 import BeautifulSoup
from shapely.geometry import Point

## Read-in

In [2]:
df = pd.read_csv('LISF_Nov_2023.csv')

In [3]:
# Make sure to update this
first_month = 11
current_month = 'November'

In [4]:
len(df)

616

In [5]:
# Define the regex pattern to split the text
pattern = r'(\d+-\d+-\d+-\d+-\d+)\s(.*)'

# Apply regex and split the text into two columns
df[['PIN', 'Address']] = df['1st PIN'].str.extract(pattern)

# Remove leading/trailing whitespace from the address column
df['Address'] = df['Address'].str.strip()

In [6]:
df = df.drop(columns=['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0','1st PIN'])

## Clean, drop, and convert

In [7]:
df['Doc Recorded'] = pd.to_datetime(df['Doc Recorded'])

In [8]:
df = df.drop_duplicates()

## Data Stuff

In [9]:
# # Count number of earliest month dates
# sep_dates_count = len(df.loc[df['Doc Recorded'].dt.month == first_month])

# print(f'Number of September dates: {sep_dates_count}')

In [10]:
df['1st Grantor'] = df['1st Grantor'].fillna('NA')
df['1st Grantee'] = df['1st Grantee'].fillna('NA')

In [11]:
municipal_authority = ['city', 'town', 'municipality', 'village','transit auth','department of transp']  # list of municipal authority keywords
df = df[~df['1st Grantor'].str.contains('|'.join(municipal_authority), case=False) & 
        ~df['1st Grantee'].str.contains('|'.join(municipal_authority), case=False)]

In [12]:
# create new column and assign colors based on month
df['COLOR'] = df['Doc Recorded'].apply(lambda x: 'red' if x.month == first_month else 'red' if x.month == second_month else 'blue' if x.month == third_month else '')

## Get mortgage amounts

In [13]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'TE': 'Trailers'
}

In [14]:
def mortgage_url_snagger(URL, headers):
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    for link in soup.find_all('a', href=True):
        if link['href'].startswith('/Document/Detail'):
            mortgage_url = 'https://crs.cookcountyclerkil.gov' + link['href']
            return mortgage_url

In [15]:
def mortgage_consi_snagger(URL, headers):
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find all tr tags
    trs = soup.find_all('tr')

    # Loop through each tr tag and look for the td tag containing 'Consideration Amount' label
    for tr in trs:
        td = tr.find('th', text='Consideration Amount:')
        if td:
            # If the td tag is found, get the next td tag containing the amount
            amount_td = td.find_next_sibling('td')
            if amount_td:
                # Print the amount
                amount = amount_td.text.strip()
                return amount
            else:
                return 'not found'

In [16]:
df['mortgage_urls'] = df['deed_urls'].apply(lambda x: mortgage_url_snagger(x, headers))

In [17]:
df['mortgage_amount'] = df['mortgage_urls'].apply(lambda x: mortgage_consi_snagger(x, headers) if x is not None else None)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_9193/2390556658.py:9: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  td = tr.find('th', text='Consideration Amount:')


In [18]:
df[['mortgage_urls','mortgage_amount']] = df[['mortgage_urls','mortgage_amount']].fillna('NA')

## Geocode

In [19]:
df['geo_address'] = df['Address'] + ' Cook County, IL'

In [20]:
len(df)

353

In [21]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [22]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['geo_address'].apply(geocode)

In [23]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## HTML Popup Formatter

In [24]:
df.columns

Index(['View Doc', 'Doc Number', 'Doc Recorded', 'Doc Executed', 'Doc Type',
       'Consi. Amt.', '1st Grantor', '1st Grantee', 'Assoc. Doc#', 'deed_urls',
       'PIN', 'Address', 'COLOR', 'mortgage_urls', 'mortgage_amount',
       'geo_address', 'geocoded', 'lat', 'lon'],
      dtype='object')

In [25]:
def popup_html(row):
    grantor = row['1st Grantor']
    grantee = row['1st Grantee']
    PIN = row['PIN']
    Address = row['Address']
    mortgage_amount = row['mortgage_amount']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Lender: </strong>{}'''.format(grantor) + '''<br>
    <strong>Borrower: </strong>{}'''.format(grantee) + '''<br>
    <strong>PIN: </strong>{}'''.format(PIN) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>Mortgage Amount: </strong>{}'''.format(mortgage_amount) + '''<br>
    </html>
    '''
    return html

In [26]:
import folium
from folium.plugins import MarkerCluster
import numpy as np

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Cook County Pending Foreclosures')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{current_month}')

### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=9.5, tiles=None)

# Create two FeatureGroups for different color pins
fg_red = folium.FeatureGroup(name=f'{current_month}') ## 

for index, row in df.iterrows():
    lat = row['lat']
    lon = row['lon']
    color = row['COLOR']
    if pd.notnull(lat) and pd.notnull(lon) and color == 'red':
        marker = folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            fill=True,
            color=color,
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_red)
    else:
        continue

# Add the FeatureGroups to the map
fg_red.add_to(m)

folium.TileLayer('OpenStreetMap', control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
folium.TileLayer('CartoDBpositron', control=False).add_to(m) 
            
# Display map
m

In [27]:
m.save('index.html')

## Map URL Snagger

In [28]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/lis_pendens_scraper_november_2023


## Stats for story

In [29]:
df['mortgage_amount_int'] = df['mortgage_amount'].str.replace('$','')
df['mortgage_amount_int'] = df['mortgage_amount_int'].str.replace(',','')
df = df.loc[df['mortgage_amount_int'] != 'NA']
df['mortgage_amount_int'] = df['mortgage_amount_int'].astype(float).fillna(0).astype(int)

In [30]:
# Count number of first month dates
dates_count = len(df.loc[df['Doc Recorded'].dt.month == first_month])

print(f'Number of {current_month} dates: {dates_count}')

Number of November dates: 279


In [31]:
x = df.loc[df['Doc Recorded'].dt.month == first_month]

In [32]:
x.reset_index(inplace=True, drop=True)

In [33]:
x.at[123,'mortgage_amount_int']=33130000
x.at[123,'mortgage_amount']='$33,130,000'

x.at[16,'mortgage_amount_int']=173600
x.at[16,'mortgage_amount']='$173,600'

In [49]:
x.sort_values(by='mortgage_amount_int',ascending=False).iloc[0:1]

,View Doc,Doc Number,Doc Recorded,Doc Executed,Doc Type,Consi. Amt.,1st Grantor,1st Grantee,Assoc. Doc#,deed_urls,PIN,Address,COLOR,mortgage_urls,mortgage_amount,geo_address,geocoded,lat,lon,mortgage_amount_int
110,View,2332515035,2023-11-21,NaN,LIS PENDENS FORECLOSURE,NaN,CASAIL 2018-CX11 W JACKSON HOLDINGS LLC,111 W JACKSON HOLDINGS LLC,1.732129e+09,https://crs.cookcountyclerkil.gov/Document/Det...,17-16-231-010-0000,"111 W JACKSON BLVD, CHICAGO",red,https://crs.cookcountyclerkil.gov/Document/Det...,"$105,000,000.00","111 W JACKSON BLVD, CHICAGO Cook County, IL","(41.8778164, -87.631246)",41.877816,-87.631246,105000000


In [50]:
print(x['mortgage_urls'].iloc[110])

https://crs.cookcountyclerkil.gov/Document/Detail?dId=MzM5NjY4MTU1&hId=OWM3ZDYzY2I2YmJkOWQ1ZmM0YTgzZTkyMzI1NDBlYjAxMTJmYThlNGIyODU0NDI4MWYzNjM2MGYyYzUzNGVlZQ2


In [36]:
print(f'{current_month}: ${x.mortgage_amount_int.sum():,}')

November: $237,444,226


In [48]:
# October
16 + 12 + 5

33

In [46]:
x['1st Grantor'].value_counts().head(60)

1st Grantor
NEWREZ LLC                                27
US BANK TRUST NATIONAL ASSN TR            16
WILMINGTON SAV FUND SOCIETY FSB TR        14
WELLS FARGO BK NA                         12
US BANK NATIONAL ASSN TR                  12
ROCKET MTG LLC                            10
CARRINGTON MTG SERVICES LLC                7
MCLP ASSET CO INC                          7
FREEDOM MTG CORPORATION                    7
US BK NATIONAL ASSOCIATION                 5
NATIONSTAR MTG LLC                         5
PENNYMAC LOAN SERVICES LLC                 5
ASSOCIATED BK NA                           5
PHH MTG CORPORATION                        5
MORTGAGE ASSETS MGMT LLC                   4
DEUTSCHE BK NATIONAL TRUST COMPANY TR      4
BANK OF NEW YORK MELLON TR                 4
FEDERAL HOME LOAN MORTGAGE CORP TR         3
LONGBRIDGE FIN LLC                         3
UNITED COMM BANK                           2
DLJ MORTGAGE CAP INC                       2
FINANCE OF AMER REVERSE LLC                

In [39]:
x['1st Grantee'].value_counts()

1st Grantee
CHICAGO TITLE LAND TRUST CO TR    3
LOGAN WORLD LLC                   2
THOMPSON JOLI J                   1
MARSHALL STEVEN A                 1
BARRETT LESLIE L                  1
                                 ..
SCHLADER BLAIR                    1
FEDERAL PARTNERS LLC              1
CANNADY STACY                     1
JACKSON THERESA                   1
2310 W BELMONT CORPORATION        1
Name: count, Length: 276, dtype: int64

In [40]:
df.to_csv(f'{current_month}_2023_foreclosures.csv')

In [41]:
# apr.sort_values(by='mortgage_amount_int',ascending=False)

In [42]:
print(f"{current_month} median mortgage foreclosure amount: ${x['mortgage_amount_int'].median():,}")

November median mortgage foreclosure amount: $180,000.0


## Boundary Analysis

In [43]:
boundaries = gpd.read_file('Boundaries - Neighborhoods.geojson')

In [44]:
# create Point objects from lat/lon columns in df
geometry = gpd.points_from_xy(x['lon'], x['lat'])

# create geodataframe from df with Point objects as geometry
gdf_filings_x = gpd.GeoDataFrame(x, geometry=geometry)

gdf_filings_x.crs = 'EPSG:4326'
gdf_filings_x = gdf_filings_x.to_crs(boundaries.crs)

# use contains method to get count of points within each neighborhood
counts_x = gpd.sjoin(gdf_filings_x, boundaries, predicate='within').groupby('pri_neigh').size().reset_index(name='count')

In [45]:
counts_x['count'].sum()

138